In [11]:
# !pip install protobuf feast[aws,redis] pyarrow

In [ ]:
# run feast ui in console:
# feast ui -p 8081

In [12]:
# configure aws cli

In [1]:
import pandas as pd
pd.read_parquet('data/house_dataset_main.parquet').head(2)

,HouseId,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,MedHouseVal,EventTimestamp,Created
0,1,2.4792,24.0,3.454704,1.134146,2251.0,3.921603,2.000,2021-12-11 18:40:03,2022-04-12 12:04:13
1,2,3.4630,8.0,6.363636,1.166297,1307.0,2.898004,2.017,2021-12-11 18:57:30,2022-04-12 12:04:13


In [3]:
pd.read_parquet('data/house_dataset_lat_lon.parquet').head(2)

,HouseId,Latitude,Longitude,EventTimestamp,Created
0,1,34.18,-118.38,2021-12-11 18:40:03,2022-02-13 22:43:53
1,2,39.08,-121.04,2021-12-11 18:57:30,2022-02-13 22:43:53


In [1]:
cd feature_store

/home/ubuntu/mlops_workshops/feast/feature_store


In [10]:
!feast apply

In [4]:
from feast import FeatureStore

store = FeatureStore(repo_path=".")

## Offline queries

In [6]:
import feast
import pandas as pd
from feast.infra.offline_stores.file_source import SavedDatasetFileStorage

entity_df = pd.DataFrame.from_dict({"HouseId": [i for i in range(1, 1000)]})
entity_df['event_timestamp'] = pd.to_datetime('now', utc=True)

retrieval_job = store.get_historical_features(
    entity_df=entity_df,
    features=store.get_feature_service('house_service')
)

In [7]:
retrieval_job.metadata.features

['house_main_view:MedInc',
 'house_main_view:HouseAge',
 'house_main_view:AveRooms',
 'house_main_view:AveBedrms',
 'house_main_view:Population',
 'house_main_view:AveOccup',
 'house_main_view:MedHouseVal',
 'house_lat_lon_view:Latitude',
 'house_lat_lon_view:Longitude']

In [8]:
dataset = store.create_saved_dataset(
    from_=retrieval_job,
    name='merged_dataset_v1',
    # storage=SavedDatasetFileStorage('s3://dvoitekh-kubeflow/feast/data/merged_dataset_v1.parquet')
    storage=SavedDatasetFileStorage('merged_dataset_v1.parquet')
)

training_df = retrieval_job.to_df()
print(len(training_df))
training_df.head()

/home/ubuntu/.local/lib/python3.10/site-packages/feast/feature_store.py:952: RuntimeWarning: Saving dataset is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


999


,HouseId,event_timestamp,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,MedHouseVal,Latitude,Longitude
0,1,2022-06-16 07:34:53.875613+00:00,2.4792,24.0,3.454704,1.134146,2251.0,3.921603,2.000,34.18,-118.38
1,2,2022-06-16 07:34:53.875613+00:00,3.4630,8.0,6.363636,1.166297,1307.0,2.898004,2.017,39.08,-121.04
2,3,2022-06-16 07:34:53.875613+00:00,3.7500,16.0,5.768719,1.023295,1478.0,2.459235,1.473,38.68,-121.28
3,4,2022-06-16 07:34:53.875613+00:00,2.8542,34.0,3.858779,1.045802,1164.0,4.442748,1.469,34.04,-118.19
4,5,2022-06-16 07:34:53.875613+00:00,1.3375,18.0,4.567625,1.087327,2707.0,2.882854,0.596,39.13,-121.54


In [27]:
!feast materialize-incremental 2022-02-01T00:00:00

06/14/2022 08:39:45 PM INFO:Found credentials in shared credentials file: ~/.aws/credentials
Materializing 2 feature views to 2022-02-01 00:00:00+00:00 into the redis online store.

Since the ttl is 0 for feature view house_main_view, the start date will be set to 1 year before the current time.
house_main_view from 2021-06-15 20:39:45+00:00 to 2022-02-01 00:00:00+00:00:
06/14/2022 08:39:46 PM INFO:Found credentials in shared credentials file: ~/.aws/credentials
/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/parquet/arrow.py:1722: DeprecationWarning: 'ParquetDataset.partitions' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Specify 'use_legacy_dataset=False' while constructing the ParquetDataset, and then use the '.partitioning' attribute instead.
  if dataset.partitions is not None:
/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/parquet/arrow.py:1797: DeprecationWarning: 'ParquetDataset.pieces' attribute is deprecated as of pya

## Online Queries

In [28]:
online_df = store.get_online_features(
    features=[
        "house_main_view:MedInc",
        "house_main_view:HouseAge",
        "house_main_view:MedHouseVal",
        "house_lat_lon_view:Latitude",
        "house_lat_lon_view:Longitude",
    ],
    entity_rows=[
        {"HouseId": 1},
        {"HouseId": 20640},
    ]
)
online_df.to_df()

,HouseId,MedInc,MedHouseVal,HouseAge,Longitude,Latitude
0,1,2.4792,2.0,24.0,-118.379997,34.18
1,20640,NaN,NaN,NaN,NaN,NaN


In [29]:
!feast materialize-incremental 2022-06-11T00:00:00

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


06/14/2022 08:40:10 PM INFO:Found credentials in shared credentials file: ~/.aws/credentials
Materializing 2 feature views to 2022-06-11 00:00:00+00:00 into the redis online store.

house_main_view from 2022-02-01 00:00:00+00:00 to 2022-06-11 00:00:00+00:00:
06/14/2022 08:40:11 PM INFO:Found credentials in shared credentials file: ~/.aws/credentials
/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/parquet/arrow.py:1722: DeprecationWarning: 'ParquetDataset.partitions' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Specify 'use_legacy_dataset=False' while constructing the ParquetDataset, and then use the '.partitioning' attribute instead.
  if dataset.partitions is not None:
/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/parquet/arrow.py:1797: DeprecationWarning: 'ParquetDataset.pieces' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Specify 'use_legacy_dataset=False' while constructing the Parqu

In [30]:
online_df = store.get_online_features(
    features=[
        "house_main_view:MedInc",
        "house_main_view:HouseAge",
        "house_main_view:MedHouseVal",
        "house_lat_lon_view:Latitude",
        "house_lat_lon_view:Longitude",
    ],
    entity_rows=[
        {"HouseId": 1},
        {"HouseId": 20640},
    ]
)
online_df.to_df()

,HouseId,MedInc,MedHouseVal,HouseAge,Longitude,Latitude
0,1,2.4792,2.000,24.0,-118.379997,34.180000
1,20640,3.3438,3.333,13.0,-117.800003,33.689999


## Push Source

In [31]:
import datetime

store.push('main_push_source', pd.DataFrame([{
    'HouseId': 1,
    'MedInc': 1.0,
    'HouseAge': 1.0,
    'AveRooms': 1.0,
    'AveBedrms': 1.0,
    'Population': 1.0,
    'AveOccup': 1.0,
    'MedHouseVal': 1.0,
    'Latitude': 1.0,
    'Longitude': 1.0,
    'EventTimestamp': datetime.datetime.now(),
    'Created': datetime.datetime.now(),
}]))

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/conda/lib/python3.8/site-packages/feast/feature_store.py:1219: RuntimeWarning: Push source is an experimental feature. This API is unstable and it could and might change in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


In [32]:
online_df = store.get_online_features(
    features=[
        "house_main_view:MedInc",
        "house_main_view:HouseAge",
        "house_main_view:MedHouseVal",
        "house_lat_lon_view:Latitude",
        "house_lat_lon_view:Longitude",
    ],
    entity_rows=[
        {"HouseId": 1},
        {"HouseId": 20640},
    ]
)
online_df.to_df()

,HouseId,MedInc,MedHouseVal,HouseAge,Longitude,Latitude
0,1,1.0000,1.000,1.0,-118.379997,34.180000
1,20640,3.3438,3.333,13.0,-117.800003,33.689999


In [33]:
entity_df = pd.DataFrame.from_dict({"HouseId": [1]})
entity_df['event_timestamp'] = pd.to_datetime('now', utc=True)

store.get_historical_features(
    entity_df=entity_df,
    features=store.get_feature_service('house_service')
).to_df()

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/parquet/arrow.py:1722: DeprecationWarning: 'ParquetDataset.partitions' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Specify 'use_legacy_dataset=False' while constructing the ParquetDataset, and then use the '.partitioning' attribute instead.
  if dataset.partitions is not None:
/opt/conda/lib/python3.8/site-packages/dask/dataframe/io/parquet/arrow.py:1797: DeprecationWarning: 'ParquetDataset.pieces' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Specify 'use_legacy_dataset=False' while constructing the Par

,HouseId,event_timestamp,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,MedHouseVal,Latitude,Longitude
0,1,2022-06-14 20:40:51.937687+00:00,2.4792,24.0,3.454704,1.134146,2251.0,3.921603,2.0,34.18,-118.38
